In [162]:
from pymongo import MongoClient
import folium
import pandas as pd

In [163]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [164]:
db, coll = connectCollection('datlab','companies')

In [165]:
pipeline = [{'$unwind':'$offices'},
        {'$match':{'$and':[{"offices.country_code":'USA'},{"funding_rounds.raised_amount":{'$gte':1000000}},{'deadpooled_year':None},{'$or':[{'offices.latitude':{'$ne':None}},{'offices.longitude':{'$ne':None}}]},{'founded_year':{'$gte': 2008}}]}}]
query = list(coll.aggregate(pipeline))

In [166]:
df = pd.DataFrame(query)

In [167]:
data = df['offices']

In [168]:
latitude = []

for key,value in data.items():
    latitude.append(value['latitude'])

In [169]:
longitude = []

for key, value in data.items():
    longitude.append(value['longitude'])

In [170]:
df1 = pd.DataFrame(list(zip(latitude,longitude)), columns =["latitude","longitude"])

In [171]:
df =pd.concat([df, df1], axis=1)

In [172]:
coleliminar = ['_id','permalink','crunchbase_url','created_at','products','funding_rounds','acquisition','acquisitions','homepage_url','blog_url', 'blog_feed_url','twitter_username','video_embeds','screenshots','external_links','partners','founded_month','founded_day','deadpooled_year','deadpooled_month','deadpooled_day','competitions','providerships','investments', 'milestones','ipo','deadpooled_url','tag_list','alias_list','email_address','phone_number','updated_at','image','overview','relationships', 'location', 'offices']
df = df.drop(coleliminar, axis=1)


In [173]:
df.to_csv('filtered_companies.csv')

In [174]:
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["datlab"]
mycol = mydb["filtered_companies"]

In [191]:
db, coll = connectCollection('datlab','filtered_companies')

In [192]:
query = list(coll.find({'name':{'$ne': None}}))

In [182]:
type(query[0]['latitude'])

float

In [184]:
def getLocation(office):
    longitude = office['longitude']
    latitude = office['latitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [185]:
for office in query:
    value = {"$set": {'location':getLocation(office)}}
    coll.update_one(office, value)

In [186]:
for office in query:
    value = {"$unset": {"longitude":""}}
    coll.update_one(office, value)

In [188]:
for office in query:
    value = {"$unset": {"latitude":""}}
    coll.update_one(office, value)

In [193]:
query[0]

{'_id': ObjectId('5dd2f3cc9787f4a6360424ef'),
 '': 1,
 'name': 'GoingOn',
 'category_code': 'software',
 'number_of_employees': 40.0,
 'founded_year': 2008,
 'description': 'Private Academic Networks',
 'total_money_raised': '$8.5M',
 'latitude': 37.782263,
 'location': {'type': 'Point', 'coordinates': [-122.392142, 37.782263]}}

In [194]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)


In [196]:
for company in query:
    folium.Marker(company['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)

In [197]:
map_city